In [ ]:
import geopandas as gpd

In [ ]:
airports_shp = gpd.read_file("../temp/airports/ne_10m_airports.shp", layer='ne_10m_airports')

In [ ]:
filter_airports = airports_shp[["name", "type", "gps_code", "iata_code", "wikipedia", "wikidataid", "geometry" ]]

In [ ]:
filter_airports